<a href="https://colab.research.google.com/github/skrzypczykt/Poem/blob/master/Elastic_vs_Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Regularyzacja_i_graficzna_ocena_klasyfikatorow

In [22]:
import numpy as np
import pandas as pd
data = np.genfromtxt('/content/australian.dat',
                     skip_header=0,
                     skip_footer=0,
                     names=True,
                     dtype=float,
                     delimiter=' ')
df = pd.DataFrame(data)
df.columns =  [f'x{i}' for i in range(15)]

In [23]:
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14
0,0.0,22.67,7.000,2.0,8.0,4.0,0.165,0.0,0.0,0.0,0.0,2.0,160.0,1.0,0.0
1,0.0,29.58,1.750,1.0,4.0,4.0,1.250,0.0,0.0,0.0,1.0,2.0,280.0,1.0,0.0
2,0.0,21.67,11.500,1.0,5.0,3.0,0.000,1.0,1.0,11.0,1.0,2.0,0.0,1.0,1.0
3,1.0,20.17,8.170,2.0,6.0,4.0,1.960,1.0,1.0,14.0,0.0,2.0,60.0,159.0,1.0
4,0.0,15.83,0.585,2.0,8.0,8.0,1.500,1.0,1.0,2.0,0.0,2.0,100.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,1.0,31.57,10.500,2.0,14.0,4.0,6.500,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0
685,1.0,20.67,0.415,2.0,8.0,4.0,0.125,0.0,0.0,0.0,0.0,2.0,0.0,45.0,0.0
686,0.0,18.83,9.540,2.0,6.0,4.0,0.085,1.0,0.0,0.0,0.0,2.0,100.0,1.0,1.0
687,0.0,27.42,14.500,2.0,14.0,8.0,3.085,1.0,1.0,1.0,0.0,2.0,120.0,12.0,1.0


In [24]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.3, random_state=42)

In [25]:
X_train = train_set.drop(['x14'],axis=1)
X_test = test_set.drop(['x14'],axis=1)
y_train = train_set['x14']
y_test = test_set['x14']


In [26]:
X_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
178,1.0,42.17,5.040,2.0,11.0,8.0,12.750,1.0,0.0,0.0,1.0,2.0,92.0,1.0
265,1.0,22.17,12.125,2.0,8.0,4.0,3.335,0.0,1.0,2.0,1.0,2.0,180.0,174.0
352,1.0,34.08,6.500,2.0,6.0,4.0,0.125,1.0,0.0,0.0,1.0,2.0,443.0,1.0
495,1.0,25.75,0.750,2.0,8.0,5.0,0.250,1.0,0.0,0.0,0.0,2.0,349.0,24.0
408,1.0,25.25,1.000,2.0,6.0,4.0,0.500,0.0,0.0,0.0,0.0,2.0,200.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.0,29.42,1.250,2.0,9.0,4.0,1.750,0.0,0.0,0.0,0.0,2.0,200.0,1.0
106,1.0,22.25,9.000,2.0,6.0,4.0,0.085,0.0,0.0,0.0,0.0,2.0,0.0,1.0
270,1.0,23.25,1.000,2.0,8.0,4.0,0.835,1.0,0.0,0.0,0.0,1.0,300.0,1.0
435,1.0,24.58,12.500,2.0,9.0,4.0,0.875,1.0,0.0,0.0,1.0,2.0,260.0,1.0


Skalowanie cech

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Zadanie

Przeprowadziliśmy procedurę wyboru najlepszego modelu dla regresji LASSO. Przeprowadź taką samą analizę dla regresji grzbietowej i modelu Elastic-Net. Który z modeli jest najlepszy? Odpowiedź uzasadnij (10 pkt.).

##RidgeRegression

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2'])
lr_l2 = LogisticRegression()
clf_l2 = RandomizedSearchCV(lr_l2, distributions, random_state=0, refit=True)
search = clf_l2.fit(X_train_scaled, y_train)
search.best_params_

{'C': 1.5337660753031108, 'penalty': 'l2'}

In [29]:
from sklearn.metrics import accuracy_score

accuracy_l2 = accuracy_score(y_test, clf_l2.predict(X_test_scaled))

Najlepsze parametry dla regresji grzbietowej:
Współczynnik regularyzacji  **1.533**

In [30]:
distributions = dict(C=uniform(loc=0, scale=4),l1_ratio=uniform(loc=0, scale=1),penalty=['elasticnet'])
lr_elastic = LogisticRegression(solver='saga', max_iter=1000)
clf_elastic = RandomizedSearchCV(lr_elastic, distributions, random_state=0, refit=True)
search = clf_elastic.fit(X_train_scaled, y_train)
search.best_params_

{'C': 0.28414423279154777,
 'l1_ratio': 0.08712929970154071,
 'penalty': 'elasticnet'}

In [31]:
accuracy_elastic_net = accuracy_score(y_test, clf_elastic.predict(X_test_scaled))

Najlepsze parametry dla regresji elasticnet:


Współczynnik regularyzacji  **0.284144**

Współczynnik regularyzacji l1 **0.087129**

Współczynnik regularyzacji l2 **1 - 0.087129 = 0.912871**

Model skłania się ku regularyzacji l2.

#Zestawienie wyników


*Wyniki dla LASSO:*

\\
conf matrix


**83.0** 19.0 

  8.0  **98.0**

\\
accuracy


**0.87**01923076923077

In [32]:
print("Accuracy dla regresji grzbietowej:", accuracy_l2)
print("Accuracy dla elasticnet:", accuracy_elastic_net)

Accuracy dla regresji grzbietowej: 0.8695652173913043
Accuracy dla elasticnet: 0.8647342995169082


In [33]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix dla regresji grzbietowej")
confusion_matrix(y_test, clf_l2.predict(X_test_scaled))

Confusion matrix dla regresji grzbietowej


array([[102,  22],
       [  5,  78]])

In [34]:
print("Confusion matrix dla elasticnet")
confusion_matrix(y_test, clf_elastic.predict(X_test_scaled))

Confusion matrix dla elasticnet


array([[101,  23],
       [  5,  78]])

Uzyskaliśmy dwa modele o gorszym accuracy. Ta miara nie jest jednak najlepszą dla zbiorów niezbilansowanych. Model regresji grzbietowe wypadł nieznacznie lepiej od elastic net. **FP** = 22 i opdowiednio 23.